In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.utils.tutorials.cnn_utils import evaluate, load_mnist, train
from torch._tensor import Tensor
from torch.utils.data import DataLoader

init_notebook_plotting()

ModuleNotFoundError: No module named 'torchvision'

In [3]:
from ax import optimize

In [6]:
best_parameters, best_values, experiment, model = optimize(
        parameters=[
          {
            "name": "x1",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
          {
            "name": "x2",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
        ],
        # Booth function
        evaluation_function=lambda p: (p["x1"] + 2*p["x2"] - 7)**2 + (2*p["x1"] + p["x2"] - 5)**2,
        minimize=True,
    )


[INFO 03-24 16:42:35] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-24 16:42:35] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-24 16:42:35] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-10.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[-10.0, 10.0])], parameter_constraints=[]).
[INFO 03-24 16:42:35] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 03-24 16:42:35] ax.modelbridge.dispatch_utils: Calcul

In [9]:
best_parameters

{'x1': 0.9185325865691762, 'x2': 3.110823735817787}

In [13]:
from rdkit import Chem
from rdkit.Chem import Draw
from dockstring import load_target
import random

# Assuming mutate_molecule is correctly defined elsewhere
# from your_mutation_logic import mutate_molecule

# Importing necessary components from Ax
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render

# Load the target protein
target = load_target('DRD2')

# Define the initial ligand
initial_ligand = Chem.MolFromSmiles("c1ccccc1C(=O)Nc2ccccc2")

# Initialize AxClient and create an experiment with a single objective
ax_client = AxClient()
ax_client.create_experiment(
    name="docking_optimization",
    parameters=[
        {"name": "mutation_rate", "type": "range", "bounds": [0.0, 1.0], "value_type": "float"},
        {"name": "mutation_type", "type": "choice", "values": ["insert", "remove", "replace"], "value_type": "str"},
    ],
    objective_name="binding_score",
    minimize=False  # Set to False to maximize the objective
)

# Define the objective function
def objective_function(parameterization):
    mutation_rate = parameterization.get('mutation_rate')
    mutation_type = parameterization.get('mutation_type')

    # Apply mutations to the initial ligand
    mutated_mol = mutate_molecule(initial_ligand, mutation_rate, mutation_type)

    # Score the mutated molecule using dockstring
    score, _ = target.dock(Chem.MolToSmiles(mutated_mol))

    return {"binding_score": (score, 0.0)}  # Assuming score is a float. The 0.0 is the SEM (standard error of the mean).

# Run the optimization loop
for _ in range(25):  # Adjust the number of iterations as needed
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=objective_function(parameters))

# Extract the best parameters and score
best_parameters, metrics = ax_client.get_best_parameters()
best_score = metrics["binding_score"]

print(f"Best parameters: {best_parameters}")
print(f"Best binding score: {best_score}")

# Visualize the best molecule
best_mol = mutate_molecule(
    initial_ligand,
    best_parameters['mutation_rate'],
    best_parameters['mutation_type']
)
Draw.MolToFile(best_mol, 'best_molecule.png')


[INFO 03-24 16:47:13] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 03-24 16:47:13] ax.service.utils.instantiation: Created search space: S

TypeError: mutate_molecule() takes 1 positional argument but 3 were given

In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import random

def mutate_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mutation_type = random.choice(['add_functional_group', 'modify_ring', 'change_chain_length'])
    
    if mutation_type == 'add_functional_group':
        # Example: Add a hydroxyl group to a random carbon atom
        carbons = [atom.GetIdx() for atom in mol.GetAtoms() if atom.GetSymbol() == 'C']
        if carbons:
            c = random.choice(carbons)
            mol = Chem.RWMol(mol)
            oh = Chem.MolFromSmiles('O')
            mol.AddBond(c, mol.AddAtom(oh.GetAtomWithIdx(0)), Chem.BondType.SINGLE)
            mol = mol.GetMol()
    
    elif mutation_type == 'modify_ring':
        # Example: Simplified version, convert a benzene ring to a pyridine (if present)
        patterns = [Chem.MolFromSmarts('c1ccccc1')]  # A benzene ring
        for pattern in patterns:
            if mol.HasSubstructMatch(pattern):
                replacement = Chem.MolFromSmiles('c1ccncc1')  # Pyridine
                mol = AllChem.ReplaceSubstructs(mol, pattern, replacement)[0]
                break

    elif mutation_type == 'change_chain_length':
        # Example: Extend a carbon chain by one carbon atom
        # Find the first carbon with fewer than 4 neighbors to add a methyl group
        for atom in mol.GetAtoms():
            if atom.GetSymbol() == 'C' and len(atom.GetNeighbors()) < 4:
                mol = Chem.RWMol(mol)
                c = mol.AddAtom(Chem.Atom('C'))
                mol.AddBond(atom.GetIdx(), c, Chem.BondType.SINGLE)
                mol = mol.GetMol()
                break

    return Chem.MolToSmiles(mol)

# Test the mutation function
initial_smiles = "c1ccccc1C(=O)Nc2ccccc2"  # An initial ligand
mutated_smiles = mutate_molecule(initial_smiles)
print(f"Initial SMILES: {initial_smiles}")
print(f"Mutated SMILES: {mutated_smiles}")

Initial SMILES: c1ccccc1C(=O)Nc2ccccc2
Mutated SMILES: Cc1ccccc1C(=O)Nc1ccccc1


In [14]:
from rdkit import Chem
from rdkit.Chem import Draw
from dockstring import load_target
import random

# Importing necessary components from Ax
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render

# Define the mutate_molecule function to accept mutation_rate and mutation_type
def mutate_molecule(smiles, mutation_rate, mutation_type):
    mol = Chem.MolFromSmiles(smiles)
    # Placeholder for mutation logic; should be implemented based on specific criteria
    if mutation_type == 'insert':
        # For simplicity, we won't implement detailed mutation logic here
        pass  # Implement add_functional_group logic
    elif mutation_type == 'remove':
        pass  # Implement remove_functional_group logic
    elif mutation_type == 'replace':
        pass  # Implement replace_functional_group logic
    return Chem.MolToSmiles(mol)  # Return the modified SMILES string

# Load the target protein
target = load_target('DRD2')

# Define the initial ligand
initial_ligand = Chem.MolFromSmiles("c1ccccc1C(=O)Nc2ccccc2")

# Initialize AxClient and create an experiment with a single objective
ax_client = AxClient()
ax_client.create_experiment(
    name="docking_optimization",
    parameters=[
        {"name": "mutation_rate", "type": "range", "bounds": [0.0, 1.0], "value_type": "float"},
        {"name": "mutation_type", "type": "choice", "values": ["insert", "remove", "replace"], "value_type": "str"},
    ],
    objective_name="binding_score",
    minimize=False  # Set to False to maximize the objective
)

# Define the objective function
def objective_function(parameterization):
    mutation_rate = parameterization.get('mutation_rate')
    mutation_type = parameterization.get('mutation_type')

    # Apply mutations to the initial ligand
    mutated_smiles = mutate_molecule(Chem.MolToSmiles(initial_ligand), mutation_rate, mutation_type)

    # Score the mutated molecule using dockstring
    score, _ = target.dock(mutated_smiles)

    return {"binding_score": (score, 0.0)}  # Assuming score is a float. The 0.0 is the SEM.

# Run the optimization loop
for _ in range(25):  # Adjust the number of iterations as needed
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=objective_function(parameters))

# Extract the best parameters and score
best_parameters, metrics = ax_client.get_best_parameters()
best_score = metrics["binding_score"]

print(f"Best parameters: {best_parameters}")
print(f"Best binding score: {best_score}")

# Visualize the best molecule
best_mol_smiles = mutate_molecule(
    Chem.MolToSmiles(initial_ligand),
    best_parameters['mutation_rate'],
    best_parameters['mutation_type']
)
best_mol = Chem.MolFromSmiles(best_mol_smiles)
Draw.MolToFile(best_mol, 'best_molecule.png')


[INFO 03-24 16:50:13] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 03-24 16:50:13] ax.service.utils.instantiation: Created search space: S

KeyboardInterrupt: 

In [17]:
from rdkit import Chem
from rdkit.Chem import Draw, rdmolops
from dockstring import load_target
import random
from rdkit.Chem import Descriptors

# Importing necessary components from Ax
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render

def mutate_molecule(smiles, mutation_rate, mutation_type):
    mol = Chem.MolFromSmiles(smiles)
    if random.random() < mutation_rate:
        if mutation_type == 'insert':
            # Example: append a hydroxyl group; in practice, choose a suitable attachment point
            smiles += 'O'
        elif mutation_type == 'remove':
            # Simplified removal; identify removable parts for a more realistic approach
            mol = Chem.RemoveHs(mol)  # Placeholder for actual removal logic
            smiles = Chem.MolToSmiles(mol)
        elif mutation_type == 'replace':
            # Simplified replace; identify and replace specific functional groups in SMILES
            smiles = smiles.replace('Cl', 'Br')  # Example replacement
        mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol)

# Load the target protein
target = load_target('DRD2')

# Define the initial ligand
initial_ligand = Chem.MolFromSmiles("c1ccccc1C(=O)Nc2ccccc2")

# Initialize AxClient and create an experiment with a single objective
ax_client = AxClient()
ax_client.create_experiment(
    name="docking_optimization",
    parameters=[
        {"name": "mutation_rate", "type": "range", "bounds": [0.0, 1.0], "value_type": "float"},
        {"name": "mutation_type", "type": "choice", "values": ["insert", "remove", "replace"], "value_type": "str"},
    ],
    objective_name="binding_score",
    minimize=False  # Set to False to maximize the objective
)

# Define the objective function
def objective_function(parameterization):
    mutation_rate = parameterization.get('mutation_rate')
    mutation_type = parameterization.get('mutation_type')
    mutated_smiles = mutate_molecule(Chem.MolToSmiles(initial_ligand), mutation_rate, mutation_type)
    mutated_mol = Chem.MolFromSmiles(mutated_smiles)

    # Score the mutated molecule using dockstring
    score, _ = target.dock(mutated_smiles)



    return {"binding_score": (score, 0.0)}
# Run the optimization loop
for _ in range(25):  # Adjust the number of iterations as needed
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=objective_function(parameters))

# Extract the best parameters and score
best_parameters, metrics = ax_client.get_best_parameters()
best_score = metrics["binding_score"]

print(f"Best parameters: {best_parameters}")
print(f"Best binding score: {best_score}")

# Visualize the best molecule
best_mol_smiles = mutate_molecule(
    Chem.MolToSmiles(initial_ligand),
    best_parameters['mutation_rate'],
    best_parameters['mutation_type']
)
best_mol = Chem.MolFromSmiles(best_mol_smiles)
Draw.MolToFile(best_mol, 'best_molecule.png')


[INFO 03-24 16:56:24] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/home/alxfgh/miniconda3/envs/newenv/lib/python3.8/site-packages/ax/core/parameter.py:517: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "mutation_type". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 03-24 16:56:24] ax.service.utils.instantiation: Created search space: S

KeyboardInterrupt: 